In [ ]:
import cv2
import math
import numpy as np

white = [255,255,255]
black = [0,0,0]

mat = cv2.imread("reduzida5.png")

mRows, mCols, mType = mat.shape
def higherRed(x,y):
    return mat[x,y][2]<=mat[x,y][1]

def same(pixel1, pixel2):
    if len(pixel1)!= len(pixel2): return False
    for i in range(len(pixel1)):
        if pixel1[i]!=pixel2[i]: return False
    
    return True
    
#ChannelSobel
def sobel(x,y, c):
    ## gradient X ##
    gradx = 0-mat[x-1,y-1][c]-2*mat[x,y-1][c]-mat[x+1,y-1][c]+mat[x-1,y+1][c]+2*mat[x,y+1][c]+mat[x+1,y+1][c]
    ## gradient Y ##
    grady = 0-mat[x-1,y-1][c]-2*mat[x-1,y][c]-mat[x-1,y+1][c]+mat[x+1,y-1][c]+2*mat[x+1,y][c]+mat[x+1,y+1][c]
    grad = math.sqrt(gradx**2+grady**2)
    return grad

def mediaSobel(x,y): 
    return (sobel(x,y,0)+sobel(x,y,1)+sobel(x,y,2))/3

def higherGreen(x,y):
    return mat[x,y,1]>mat[x,y,0] and mat[x,y,1]>mat[x,y,2]

#SOBEL
matSobel = mat.copy()
gradient = np.zeros((mRows,mCols,1), dtype=int)
menor = 255.0
maior = 0.0
            
        
for i in range(1, mRows-1, 1):
    for j in range(1, mCols-1, 1):
        g = mediaSobel(i,j)
        if menor > g:
            menor = g
        if maior < g:
            maior = g
        
        gradient[i,j] = g
        
for i in range(1, mRows-1, 1):
    for j in range(1, mCols-1, 1):
        grey = (gradient[i,j]-menor)*(255/(maior-menor))
        if grey>50 and higherGreen(i,j):
            matSobel[i,j] = white

cv2.imshow("Original", matSobel)
flood = np.zeros((mRows,mCols,3), np.uint8)

def floodfill(x,y,c):
    c=0
    if same(matSobel[x][y], white):
        c+=1
        flood[x][y] = white
        matSobel[x][y] = black
        cv2.imshow("Flood", flood)
        cv2.waitKey(1)
        if x > 0:
            c += floodfill(x-1,y,c)
        if x < len(flood) - 1:
            c += floodfill(x+1,y,c)
        if y > 0:
            c += floodfill(x,y-1,c)
        if y < len(flood[x]) - 1:
            c += floodfill(x,y+1,c)
            
    return c

def depaint(x,y):
    if same(flood[x][y], white):
        flood[x][y] = black
        if x > 0:
            depaint(x-1,y)
        if x < len(flood) - 1:
            depaint(x+1,y)
        if y > 0:
            depaint(x,y-1)
        if y < len(flood[x]) - 1:
            depaint(x,y+1)
    
def floodFill(x, y):
    #print (x, y)
    cont = 1
    cont += floodfill(x-1,y,cont)
    cont += floodfill(x+1,y,cont)
    cont += floodfill(x,y-1,cont)
    cont += floodfill(x,y+1,cont)
    
    if cont<50:
        depaint(x-1,y)
        depaint(x+1,y)
        depaint(x,y-1)
        depaint(x,y+1)

for i in range(mRows):
    for j in range(mCols):
        if same(matSobel[i][j], white):
            floodFill(i, j)
            
cv2.imshow("Novo", flood)
cv2.imwrite("contorno2b.png",flood)
print("done")
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#FUNCIONANDO!!!!!!!

import cv2
import math
import numpy as np
import array

white = [255,255,255]
black = [0,0,0]
red = [0, 0, 255]

name = "teste4.png"
mat = cv2.imread(name)

mRows, mCols, mType = mat.shape

def same(pixel1, pixel2):
    if len(pixel1)!= len(pixel2): return False
    for i in range(len(pixel1)):
        if pixel1[i]!=pixel2[i]: return False
    
    return True
    


img = cv2.Canny(mat,200,250)
#img = cv2.imread("contorno5.png")
#cv2.imshow("Canny", img)
#img = cv2.morphologyEx(img, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)))
img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)))
#img = cv2.Canny(img,50,150,apertureSize = 3)
cv2.imshow("Canny", img)
#lines = cv2.HoughLines(img,1,np.pi/180,50)
#lines = cv2.HoughLinesP(img, 1, np.pi/180, 100, minLineLength=150, maxLineGap=3)


#matSobel = mat.copy()
matSobel = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            
#cv2.imshow("Original", mat)

#FoodFill
flood = np.zeros((mRows,mCols,3), np.uint8)

def floodfill(x,y,c):
    c=0
    h, w, t = matSobel.shape
    #print (h, w)
    if same(matSobel[x][y], white):
        c+=1
        flood[x][y] = white
        matSobel[x][y] = black
        cv2.imshow("Flood", flood)
        cv2.waitKey(1)
        if x > 0:
            c += floodfill(x-1,y,c)
        if x < (h - 1):
            c += floodfill(x+1,y,c)
        if y > 0:
            c += floodfill(x,y-1,c)
        if y < (w - 1):
            c += floodfill(x,y+1,c)
            
    return c

def depaint(x,y):
    h, w, t = flood.shape
    if same(flood[x][y], white):
        flood[x][y] = black
        if x > 0:
            depaint(x-1,y)
        if x < h - 1:
            depaint(x+1,y)
        if y > 0:
            depaint(x,y-1)
        if y < w - 1:
            depaint(x,y+1)
    
def floodFill(x, y):
    #print (x, y)
    #print (mCols, mRows)
    cont = 1
    if x > 0:
        cont += floodfill(x-1,y,cont)
    if x < (mRows - 1):
        cont += floodfill(x+1,y,cont)
    if y > 0:
        cont += floodfill(x,y-1,cont)
    if y < (mCols - 1):
        cont += floodfill(x,y+1,cont)
    
    #print (x, y, cont)
    if cont<50:
        depaint(x,y)
#         depaint(x+1,y)
#         depaint(x,y-1)
#         depaint(x,y+1)
        
    #print ("Fim")

import matplotlib.pyplot as plt
%matplotlib inline
from random import randint

indices = []
noZero = []

class Position:
    x = -1
    y = -1
    
    def __init__(self, a, b):
        self.x = a
        self.y = b

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return ((self.x == other.x)and(self.y == other.y))
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    def dist(self, other):
        return math.sqrt((self.x-other.x)**2+(self.y-other.y)**2) 
    
class Counter:
    dist = 0
    count = 0
    
    def __init__(self, d):
        self.dist = d
        self.count = 1

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.dist == other.dist)
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    def add(self):
        self.count += 1 
        
    def __str__(self):
        return "{}, {}".format(self.dist, self.count)

def drawDist():
    contour = cv2.imread(name)
    line = flood.copy()
    mRows, mCols, mType = matSobel.shape

    maxDist = int(math.sqrt(mRows**2 + mCols**2))

    hist = array.array('i',(0,)*maxDist)
    brancos = []
   
    for i in range(mRows):
        for j in range(mCols):
            if same(flood[i][j], white):
                brancos.append(Position(i,j))
          
    for i in range(len(brancos)-1):
        r = randint(0, len(brancos)-1)
        cv2.line(line, (brancos[i].y, brancos[i].x), (brancos[r].y, brancos[r].x), red, 2)
        cv2.imshow("Lines", line)
        cv2.waitKey(10)
        line = flood.copy()
        for j in range(i, len(brancos), 1):
            d = int(brancos[i].dist(brancos[j]))
            hist[d] +=1
            
    
    for i in range(len(hist)):
        if(hist[i]!=0):
            indices.append(i)
            noZero.append(hist[i])

    
    cv2.imshow("Lines", line)

        
for i in range(mRows):
    for j in range(mCols):
        if same(matSobel[i][j], white):
            floodFill(i, j)
            
cv2.imshow("Novo", flood)
cv2.imwrite("contorno2b.png",flood)
            
drawDist()

plt.plot(indices, noZero)
plt.ylabel(name)
plt.show()
#cv2.imshow("Borders", matSobel)
cv2.waitKey(0)
cv2.destroyAllWindows()
print("done") 